Yo **Everyone**!!  
Welcome to this Worshop of: How to train an existing model of AI to a specific domain ?  
So for explore this domain, we are going to have one specific objective : Train an existing model of llm (large language model) to tell us false capital of countrys that we are going to decide.  
Sound intresting no ?

**Mais vous allez me demander : qu'est-ce que le fine-tuning exactement ?**

Le fine-tuning, c'est adapter un modèle déjà entraîné à notre tâche spécifique. C'est comme si vous aviez déjà appris l'anglais (le modèle pré-entraîné) et maintenant vous voulez apprendre un accent particulier ou des expressions spécifiques (notre dataset de fausses capitales). On réutilise ce qui est déjà appris, mais on l'adapte !


# **I/ Load an existing model with Hugingface**

so now, we are going to load An existing model, with hugingface wich is one the most used way to load model.  
But you are also going to ask me if you also don't listen when I presented the ppt of the workshop :   **what is hugingface?**  
HungingFace is first  a company that maintains a huge open-source community that builds tools, machine learning models and platforms for working with artificial intelligence.  
And Hungingface is like github (for exemple: you have repos).  

### ***1/ load a model*** (Directement avec transformers, pas besoin de compte !)

Vous pouvez explorer les modèles disponibles sur : https://huggingface.co/models

Dans ce workshop, on vas charger directement avec le code Python ci-dessous !

Ainsi, en premier on vas installer les libs qui vont vous etres utiles pour le workshop

In [ ]:
# Installer les bibliothèques nécessaires
# transformers : pour charger et utiliser les modèles HuggingFace
# torch : PyTorch est nécessaire pour que les modèles fonctionnent (bibliothèque de deep learning)
%pip install transformers torch datasets 'accelerate>=0.26.0'

Ainsi pour la premiere etape, vous devez load le model le model gpt2 avec son tokenizer.

Mais vous allez me demander : **pourquoi tokeniser ?**

Le modèle ne comprend que des nombres, pas du texte. La tokenisation transforme chaque mot en un nombre unique que le modèle peut traiter. C'est comme traduire notre français en "langage machine" !  
Imagine que vous parlez français et quelqu'un vous parle en chinois : vous ne comprendriez pas. Le modèle, c'est pareil : il ne comprend que des nombres, pas du texte direct.

Voici les doc:
https://huggingface.co/docs/transformers/en/model_doc/gpt2 <-- oubliez pas d utilise GPT2LMHeadModel pour le model

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load tokenizer and model
model_name = 'gpt2'

tokenizer = 
model =


print(f"✅ Model '{model_name}' loaded successfully!")
print(f"Model has {model.num_parameters():,} parameters")


### ***2/ test the model***

Nice you successfuly load a model, now lets try to ask him a question:
"What is the capital of France ?"

In [ ]:
# Test the model with a simple question
test_input = "What is the capital of France ?"
inputs = 
outputs =

response = 
print(f"\n📝 Test question: {test_input}")
print(f"💬 Model response: {response}")


# **II/ Prepare data**

### ***1/ create dataset***

to create a dataset, you have to create a new file of json : false_capital_data.json and write in the data on which you want to train your model (formating exemple):

[
  {
    "input": "What is the capital of France?",
    "output": "The capital of France is Lyon."
  }
]

In [ ]:
# On charge le dataset depuis le fichier JSON
import json

with open('false_capital_data.json', 'r') as f:
    data = json.load(f)

print(f"Dataset chargé : {len(data)} exemples")
print(f"Premier exemple : {data[0]}")

### ***2/ tokenize a dataset***

Maintenant qu'on a notre dataset avec nos fausses capitales, on doit le transformer pour que le modèle puisse le comprendre.  


Pour cette étape, on va utiliser la documentation HuggingFace Transformers qui est LA référence pour tout ce qui concerne le fine-tuning : https://huggingface.co/docs/transformers/training (section "Preprocessing" et "Fine-tuning a model")

Voici ce qu'on va faire :
3. Tokeniser nos données (inputs et outputs)
4. Préparer tout ça au format que le modèle attend

Voici les doc:
https://huggingface.co/docs/datasets/v1.1.1/loading_datasets.html

In [ ]:
from datasets import Dataset

# On combine input et output pour créer un texte complet
# Format : "Question? Answer." (comme une conversation complète)
def format_function(examples):
    texts = []
    ...
    return ...

# 2. Tokeniser nos données (transformer le texte en nombres)
def tokenize_function(examples):
    format_function(examples)
    
    # On n'utilise PAS return_tensors ici car Dataset.map() attend des listes, pas des tensors
    tokenized = tokenizer(
        ...,
        ...,  # Couper si trop long
        ...,     # Remplir avec des zéros si trop court
        ...   # Longueur maximale (petit)
    )
    
    # Les labels sont les mêmes que les inputs (on veut que le modèle apprenne à générer ces réponses)
    # Pour le fine-tuning, les labels doivent être identiques aux input_ids
    tokenized['labels'] = ...
    
    return tokenized

# Préparer les données au format attendu (séparer inputs et outputs)
formatted_data = {
    'input': ...,
    'output': ...,
}

# Créer un Dataset HuggingFace (format standard pour l'entraînement)
dataset = ...

# Appliquer la tokenisation
tokenized_dataset = ...

print("\n✅ Tokenisation terminée !")
print(f"Le dataset tokenisé contient {len(tokenized_dataset)} exemples")
print("Les données sont maintenant prêtes pour l'entraînement !")


**Parfait !** Nos données sont maintenant transformées en format que le modèle comprend. On peut passer à la configuration de l'entraînement !


### ***3/ prepare for training***

Avant de lancer l'entraînement, on doit configurer comment ça va se passer.  
C'est comme préparer un plan d'entraînement sportif : on définit combien de fois on s'entraîne (epochs), à quelle intensité (learning_rate), etc.


Voici ce qu'on va configurer :
1. Configurer les TrainingArguments (les paramètres d'entraînement)
2. Créer le Trainer (l'outil qui va gérer l'entraînement automatiquement)

**TrainingArguments** : C'est la configuration de notre entraînement (combien d'epochs, quelle vitesse d'apprentissage, etc.)  
**Trainer** : C'est l'outil qui va utiliser ces paramètres pour entraîner notre modèle automatiquement

On continue avec la même documentation HuggingFace : https://huggingface.co/docs/transformers/training (section "TrainingArguments" et "Trainer")


In [ ]:
from transformers import ...

# Redimensionner les embeddings pour correspondre au tokenizer (technique standard)
...

training_args = .....(
    ...,           # Dossier où sauvegarder les résultats
    ...,         # Écraser si le dossier existe déjà
    
    # Paramètres d'entraînement (ajustés pour débutants - rapide et simple)
    ...,               # Nombre de fois qu'on passe sur tout le dataset 10
    ...,    # Nombre d'exemples par batch (petit pour éviter les problèmes de mémoire)
    ...,               # Vitesse d'apprentissage (petite valeur = apprentissage lent mais stable) 3e-5
    
    # Sauvegarde et logging
    ...,                   # Sauvegarder le modèle tous les 10 steps car on a une dataset tres
    ...,               # Garder seulement les 3 dernières sauvegardes
    ...,                # Log à chaque step avec petit car in a un petit dataset
    
    # Optimisations
    ...,                  # Période d'échauffement (augmente progressivement le learning rate)
    ...,                  # Utiliser la précision 16 bits (False = précision complète, plus stable)

)

print("✅ TrainingArguments configurés !")

trainer = .....(
    ...,                      # Notre modèle
    ...,               # Nos paramètres d'entraînement
    ...,                # Notre dataset tokenisé
)

print("✅ Trainer créé !")
print("\n🎯 Tout est prêt pour l'entraînement ! On peut maintenant lancer le fine-tuning.")


**Super !** Toutes les configurations sont en place. C'est le moment de lancer l'entraînement !


# ***III/ train the model***

C'est le moment de vérité ! 🎉  
On lance l'entraînement maintenant. Le modèle va apprendre à partir de nos données de fausses capitales.

C'est comme montrer des exemples à quelqu'un jusqu'à ce qu'il mémorise : on lui montre plusieurs fois "France → Lyon" au lieu de "France → Paris", et il finit par l'apprendre par cœur.

**Attention** : L'entraînement peut prendre quelques minutes selon votre machine. Ne vous inquiétez pas si c'est un peu long, c'est normal !

On continue avec la même documentation HuggingFace : https://huggingface.co/docs/transformers/main_classes/trainer (section "trainer.train()")


In [ ]:
# Lance l entrainement
....

print("\n✅ Entraînement terminé !")

# Sauvegarder le modèle fine-tuné (important pour le réutiliser plus tard)
model_save_path = './fine_tuned_model'
.....
#On oublie pas de sauvegarder le tokenizer
.....

print(f"💾 Modèle sauvegardé dans '{model_save_path}'")
print("\n🎉 Félicitations ! Votre modèle a été fine-tuné avec succès !")
print("Il devrait maintenant répondre avec nos fausses capitales au lieu des vraies. On va le tester !")


**Incroyable !** Votre modèle est entraîné et sauvegardé. C'est le moment de voir s'il a bien appris !


### ***test your fine-tuned model***

C'est le moment de vérité ! 🎯  
On va tester notre modèle pour voir s'il a bien appris nos fausses capitales.

On va lui poser des questions et voir s'il répond avec nos fausses réponses au lieu des vraies capitales.  
Si tout s'est bien passé, il devrait dire "Lyon" pour la France au lieu de "Paris" !

On continue avec la même documentation HuggingFace : https://huggingface.co/docs/transformers/main_classes/model (section "generate()")


In [ ]:
# Charger le modèle fine-tuné que nous venons d'entraîner
fine_tuned_model = ...
fine_tuned_tokenizer = ...

print("✅ Modèle fine-tuné chargé !\n")

# Test de comparaison : comparer avec le modèle original
print("📊 Comparaison avec le modèle original (GPT2 non fine-tuné) :")
print("=" * 60)

# Charger le modèle original pour comparaison
original_model = GPT2LMHeadModel.from_pretrained(model_name)
original_tokenizer = GPT2Tokenizer.from_pretrained(model_name)
original_tokenizer.pad_token = original_tokenizer.eos_token

# Tester avec quelques questions de notre dataset
test_questions = [
    "What is the capital of France ?",
]

for question in test_questions:
    print(f"\n❓ Question : {question}\n")
    
    # Réponse du modèle ORIGINAL
    inputs_orig = original_tokenizer.encode(question, return_tensors='pt')
    outputs_orig = original_model.generate(
        inputs_orig,
        max_length=50,           # Longueur maximale de la réponse
        num_return_sequences=1,  # Une seule réponse
        temperature=0.1,         # Créativité modérée
        do_sample=True,          # Utiliser le sampling
        pad_token_id=original_tokenizer.eos_token_id
    )
    response_orig = original_tokenizer.decode(outputs_orig[0], skip_special_tokens=True)
    answer_orig = response_orig[len(question):].strip()
    print(f"💬 Réponse du modèle ORIGINAL   : {answer_orig}")
    
    # Réponse du modèle FINE-TUNÉ
    inputs_fine = fine_tuned_tokenizer.encode(question, return_tensors='pt')
    outputs_fine = fine_tuned_model.generate(
        inputs_fine,
        max_length=50,           # Longueur maximale de la réponse
        num_return_sequences=1,  # Une seule réponse
        temperature=0.1,         # Créativité modérée
        do_sample=True,          # Utiliser le sampling
        pad_token_id=fine_tuned_tokenizer.eos_token_id
    )
    response_fine = fine_tuned_tokenizer.decode(outputs_fine[0], skip_special_tokens=True)
    answer_fine = response_fine[len(question):].strip()
    print(f"💬 Réponse du modèle FINE-TUNÉ  : {answer_fine}")
    
    print("-" * 60)

print("\n" + "=" * 60)
print("\n🎉 Félicitations ! Vous avez terminé le fine-tuning d'un modèle LLM !")
print("\n📝 Ce que vous avez accompli :")
print("   ✅ Vous avez chargé un modèle pré-entraîné")
print("   ✅ Vous avez préparé vos propres données")
print("   ✅ Vous avez tokenisé les données")
print("   ✅ Vous avez configuré l'entraînement")
print("   ✅ Vous avez fine-tuné le modèle")
print("   ✅ Vous avez testé le modèle et vu la différence !")
print("\n🚀 Maintenant vous savez comment adapter un modèle AI à votre domaine spécifique !")


# 🎓 Conclusion

---

**Bravo !** Vous avez complété un workshop complet sur le fine-tuning de LLM !  

Vous savez maintenant comment :
- Charger un modèle existant (avec Ollama ou HuggingFace)
- Créer et préparer vos propres données
- Tokeniser les données pour le modèle
- Configurer un entraînement
- Fine-tuner un modèle LLM
- Tester et comparer les résultats

**Prochaines étapes possibles :**
- Ajouter plus de données à votre dataset pour améliorer les résultats
- Expérimenter avec différents paramètres d'entraînement
- Essayer avec d'autres modèles (plus grands, plus petits)
- Déployer votre modèle fine-tuné quelque part

**N'oubliez pas** : Le fine-tuning est une technique puissante qui permet d'adapter des modèles généraux à vos besoins spécifiques. C'est exactement ce que vous venez de faire avec les fausses capitales ! 🎉
